In [23]:
import pandas as pd
import numpy as np
import os
import datetime as dt
import calendar
from matplotlib import pyplot as plt
import scipy.stats as stats
import glob

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

from mpl_toolkits.mplot3d import Axes3D  #3Dplot
import seaborn as sns
import japanize_matplotlib


from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

from matplotlib.ticker import ScalarFormatter
import matplotlib.ticker as ptick ##これが必要！

from matplotlib.ticker import MaxNLocator

In [2]:
files = glob.glob('../../../卒論関連書類/データ/king_history/king_history1/*')
version = len(files)-1
df_king =pd.read_csv(f'../../../卒論関連書類/データ/king_history/king_history1/king{version}.csv',converters={'DateTime':pd.to_datetime})


In [5]:

df1 = df_king[['DateTime','gap_price[intra-spot](円/kWh)','太陽光絶対値誤差合算(実績-想定)(kWh)']].dropna()

In [20]:
year=2022
percent =50
df = df1[(df1['DateTime'].dt.year==year)&(df1['DateTime'].dt.hour>=9)&(df1['DateTime'].dt.hour<=16)]


df.loc[:,'judge'] =0

df['gap_price[intra-spot](円/kWh)'] =abs(df['gap_price[intra-spot](円/kWh)'])
judge_line = np.percentile(df['gap_price[intra-spot](円/kWh)'],percent)

df.loc[df['gap_price[intra-spot](円/kWh)']>judge_line,['judge']] =1


C:\Users\Annex Rakyu\AppData\Local\Temp\ipykernel_6484\2705353754.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'judge'] =0
C:\Users\Annex Rakyu\AppData\Local\Temp\ipykernel_6484\2705353754.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gap_price[intra-spot](円/kWh)'] =abs(df['gap_price[intra-spot](円/kWh)'])


In [21]:
df

,DateTime,gap_price[intra-spot](円/kWh),太陽光絶対値誤差合算(実績-想定)(kWh),judge
66673,2022-01-01 09:00:00,5.91,250631.932962,1
66674,2022-01-01 09:30:00,3.05,184973.486658,0
66675,2022-01-01 10:00:00,2.62,284123.186423,0
66676,2022-01-01 10:30:00,1.51,298702.090738,0
66677,2022-01-01 11:00:00,7.85,397339.462726,1
...,...,...,...,...
79740,2022-09-30 14:30:00,3.08,197858.191143,0
79741,2022-09-30 15:00:00,4.59,266132.919788,1
79742,2022-09-30 15:30:00,4.62,227144.453454,1
79743,2022-09-30 16:00:00,4.29,260739.104841,1


In [55]:

name='太陽光絶対値誤差合算(実績-想定)(kWh)'
change_num =10**4
rechange_num =3
start_time =5
endtime=19

x_label ='$10^3$kWh'

for year in range(2018,2023):
        title =f'{year} 太陽光発電誤差'
        percent =50
        df = df1[(df1['DateTime'].dt.year==year)&(df1['DateTime'].dt.hour>=start_time)&(df1['DateTime'].dt.hour<=endtime)]


        df.loc[:,'judge'] =0

        df['gap_price[intra-spot](円/kWh)'] =abs(df['gap_price[intra-spot](円/kWh)'])
        judge_line = np.percentile(df['gap_price[intra-spot](円/kWh)'],percent)

        df.loc[df['gap_price[intra-spot](円/kWh)']>judge_line,['judge']] =1



        value0 = df[df['judge']==0][name].values/change_num
        value1 = df[df['judge']==1][name].values/change_num


        plt.rcParams['font.family'] = 'Times New Roman'
        plt.rcParams['font.size'] =25

        data=[value0,value1]

        fig =plt.figure(figsize=(8,8))
        ax1 = fig.subplots()
        ax1.yaxis.set_major_locator(MaxNLocator(5))
        ax1.boxplot(data,whis=(0,100),vert=True,  # 横向きにする
                patch_artist=True,  # 細かい設定をできるようにする
                widths=0.5,  # boxの幅の設定
                boxprops=dict(facecolor='gray',  # boxの塗りつぶし色の設定
                        color='black', linewidth=1),  # boxの枠線の設定
                medianprops=dict(color='black', linewidth=1),  # 中央値の線の設定
                whiskerprops=dict(color='black', linewidth=1),  # ヒゲの線の設定
                capprops=dict(color='black', linewidth=1),  # ヒゲの先端の線の設定
                )
        ax1.set_xticks(ax1.get_xticks())
        ax1.set_xticklabels(['Negative','Positive'])
        ax1.set_ylabel(x_label)

        y_ticks =np.array(ax1.get_yticks())
        ax1.set_yticks(y_ticks)
        ax1.set_yticklabels(np.round(y_ticks))
        # ax1.set_xlabel('Year')
        plt.title(title
        ,fontname='MS Mincho'
        )
        # value0=value0*change_num
        # value1 =value1*change_num

        num_round = 4
        # print(year,name)
        # print('平均値','negative',round(np.mean(value0),num_round),' ','positive',round(np.mean(value1),num_round))
        # print('中央値','negative',round(np.median(value0),num_round),' ','positive',round(np.median(value1),num_round))
        # print('標準偏差','negative',round(np.std(value0,ddof=1),num_round),' ','positive',round(np.std(value1,ddof=1),num_round))
        # print('四分位範囲','negative',round(np.percentile(value0,75)-np.percentile(value0,25),num_round),' ','positive',round(np.percentile(value1,75)-np.percentile(value1,25),num_round))
        savename = title.replace(' ','')
        dpi_n=500
        plt.savefig(f'../../../卒論関連書類/画像/4章画像_new/要因箱ひげ/太陽光発電＿夜を除く/{savename}.png',bbox_inches='tight',dpi=dpi_n)
        plt.close()




        value0=value0*change_num/10**rechange_num
        value1 =value1*change_num/10**rechange_num

        num_round = 1
        # print(year,name)
        # print('平均値','negative',round(np.mean(value0),num_round),' ','positive',round(np.mean(value1),num_round))
        # print('中央値','negative',round(np.median(value0),num_round),' ','positive',round(np.median(value1),num_round))
        # print('標準偏差','negative',round(np.std(value0,ddof=1),num_round),' ','positive',round(np.std(value1,ddof=1),num_round))
        # print('四分位範囲','negative',round(np.percentile(value0,75)-np.percentile(value0,25),num_round),' ','positive',round(np.percentile(value1,75)-np.percentile(value1,25),num_round))
        df_show = pd.DataFrame(columns=['平均値','中央値','標準偏差','四分位範囲'],index=['negative','positive'])
        df_show['平均値']=[round(np.mean(value0),num_round),round(np.mean(value1),num_round)]
        df_show['中央値']=[round(np.median(value0),num_round),round(np.median(value1),num_round)]
        df_show['標準偏差']=[round(np.std(value0,ddof=1),num_round),round(np.std(value1,ddof=1),num_round)]
        df_show['四分位範囲']=[round(np.percentile(value0,75)-np.percentile(value0,25),num_round),round(np.percentile(value1,75)-np.percentile(value1,25),num_round)]
        display(df_show)


        X=value0
        Y=value1
        print('')
        print(year,name)       
        t_value, p_value = stats.ttest_ind(X, Y, alternative='two-sided',equal_var=False)

        
        if p_value < 0.025/26:
            print("t_value:", t_value)
            print("p_value:", p_value)
    
            print(f"p = {p_value:.3f} のため、帰無仮説が棄却されました。ある より ない　は大きい{year}")
        else:
            # print(f"{p_value:.3f} のため、帰無仮説が採択されました。あるとないに差はありません")
            pass


C:\Users\Annex Rakyu\AppData\Local\Temp\ipykernel_6484\3525340292.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'judge'] =0
C:\Users\Annex Rakyu\AppData\Local\Temp\ipykernel_6484\3525340292.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gap_price[intra-spot](円/kWh)'] =abs(df['gap_price[intra-spot](円/kWh)'])


,平均値,中央値,標準偏差,四分位範囲
negative,732.9,562.5,681.5,822.9
positive,712.1,555.2,678.7,895.5



2018 太陽光絶対値誤差合算(実績-想定)(kWh)


C:\Users\Annex Rakyu\AppData\Local\Temp\ipykernel_6484\3525340292.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'judge'] =0
C:\Users\Annex Rakyu\AppData\Local\Temp\ipykernel_6484\3525340292.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gap_price[intra-spot](円/kWh)'] =abs(df['gap_price[intra-spot](円/kWh)'])


,平均値,中央値,標準偏差,四分位範囲
negative,730.5,561.1,717.0,937.4
positive,692.8,520.3,681.3,863.7


...

,平均値,中央値,標準偏差,四分位範囲
negative,638.0,515.3,573.7,763.8
positive,536.0,406.3,545.4,764.7



2021 太陽光絶対値誤差合算(実績-想定)(kWh)
t_value: 9.534880460391038
p_value: 1.8186773953254323e-21
p = 0.000 のため、帰無仮説が棄却されました。ある より ない　は大きい2021


C:\Users\Annex Rakyu\AppData\Local\Temp\ipykernel_6484\3525340292.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'judge'] =0
C:\Users\Annex Rakyu\AppData\Local\Temp\ipykernel_6484\3525340292.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['gap_price[intra-spot](円/kWh)'] =abs(df['gap_price[intra-spot](円/kWh)'])


,平均値,中央値,標準偏差,四分位範囲
negative,536.8,351.4,582.8,758.3
positive,703.4,602.9,583.8,762.0



2022 太陽光絶対値誤差合算(実績-想定)(kWh)
t_value: -12.92267298339701
p_value: 7.845696454211986e-38
p = 0.000 のため、帰無仮説が棄却されました。ある より ない　は大きい2022


In [ ]:
# 検定


In [ ]:
       value0=value0*change_num/10**rechange_num
       value1 =value1*change_num/10**rechange_num
       
       num_round = 1
       print(year,target[0])
       # print('平均値','negative',round(np.mean(value0),num_round),' ','positive',round(np.mean(value1),num_round))
       # print('中央値','negative',round(np.median(value0),num_round),' ','positive',round(np.median(value1),num_round))
       # print('標準偏差','negative',round(np.std(value0,ddof=1),num_round),' ','positive',round(np.std(value1,ddof=1),num_round))
       # print('四分位範囲','negative',round(np.percentile(value0,75)-np.percentile(value0,25),num_round),' ','positive',round(np.percentile(value1,75)-np.percentile(value1,25),num_round))
       df_show = pd.DataFrame(columns=['平均値','中央値','標準偏差','四分位範囲'],index=['negative','positive'])
       df_show['平均値']=[round(np.mean(value0),num_round),round(np.mean(value1),num_round)]
       df_show['中央値']=[round(np.median(value0),num_round),round(np.median(value1),num_round)]
       df_show['標準偏差']=[round(np.std(value0,ddof=1),num_round),round(np.std(value1,ddof=1),num_round)]
       df_show['四分位範囲']=[round(np.percentile(value0,75)-np.percentile(value0,25),num_round),round(np.percentile(value1,75)-np.percentile(value1,25),num_round)]
       return(df_show)

In [33]:
# 指標の値をｄｆで出す関数
def Caliculater(name,start_year,end_year,title,x_label,change_num,rechange_num):

       
       target =[name]
       cols = ['DateTime', 'gap_price[intra-spot](円/kWh)']+target

       df_select = df_king[cols].dropna()

       re_col = ['DateTime', '価格差 (時間前市場 - スポット市場) (Yen/kWh)'
              ]+target
       df_select.columns = re_col
       # df_select['時間前市場取引量 (kWh)'] =df_select['時間前市場取引量 (kWh)']*1000/2


       percent =50
       # これは各年

       # year=2022
       start_point = dt.datetime(year,1,1,0,0)
       end_point = dt.datetime(year,12,31,23,30)
       df = df_select[(df_select['DateTime']>=start_point)&(df_select['DateTime']<=end_point)].dropna()

       df['judge'] =0

       df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'] =abs(df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'])
       judge_line = np.percentile(df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'],percent)

       df.loc[df['価格差 (時間前市場 - スポット市場) (Yen/kWh)']>judge_line,['judge']] =1

       name=re_col[-1]
       value0 = df[df['judge']==0][name].values/change_num
       value1 = df[df['judge']==1][name].values/change_num




       value0=value0*change_num/10**rechange_num
       value1 =value1*change_num/10**rechange_num
       
       num_round = 1
       print(year,target[0])
       # print('平均値','negative',round(np.mean(value0),num_round),' ','positive',round(np.mean(value1),num_round))
       # print('中央値','negative',round(np.median(value0),num_round),' ','positive',round(np.median(value1),num_round))
       # print('標準偏差','negative',round(np.std(value0,ddof=1),num_round),' ','positive',round(np.std(value1,ddof=1),num_round))
       # print('四分位範囲','negative',round(np.percentile(value0,75)-np.percentile(value0,25),num_round),' ','positive',round(np.percentile(value1,75)-np.percentile(value1,25),num_round))
       df_show = pd.DataFrame(columns=['平均値','中央値','標準偏差','四分位範囲'],index=['negative','positive'])
       df_show['平均値']=[round(np.mean(value0),num_round),round(np.mean(value1),num_round)]
       df_show['中央値']=[round(np.median(value0),num_round),round(np.median(value1),num_round)]
       df_show['標準偏差']=[round(np.std(value0,ddof=1),num_round),round(np.std(value1,ddof=1),num_round)]
       df_show['四分位範囲']=[round(np.percentile(value0,75)-np.percentile(value0,25),num_round),round(np.percentile(value1,75)-np.percentile(value1,25),num_round)]
       return(df_show)


In [9]:
def func_hakohigePLOT(name,start_year,end_year,title,x_label,change_num):

       
       target =[name]
       cols = ['DateTime', 'gap_price[intra-spot](円/kWh)']+target

       df_select = df_king[cols].dropna()

       re_col = ['DateTime', '価格差 (時間前市場 - スポット市場) (Yen/kWh)'
              ]+target
       df_select.columns = re_col
       # df_select['時間前市場取引量 (kWh)'] =df_select['時間前市場取引量 (kWh)']*1000/2


       percent =50
       # これは各年

       # year=2022
       start_point = dt.datetime(year,1,1,0,0)
       end_point = dt.datetime(year,12,31,23,30)
       df = df_select[(df_select['DateTime']>=start_point)&(df_select['DateTime']<=end_point)].dropna()

       df['judge'] =0

       df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'] =abs(df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'])
       judge_line = np.percentile(df['価格差 (時間前市場 - スポット市場) (Yen/kWh)'],percent)

       df.loc[df['価格差 (時間前市場 - スポット市場) (Yen/kWh)']>judge_line,['judge']] =1


       value0 = df[df['judge']==0][name].values/change_num
       value1 = df[df['judge']==1][name].values/change_num


       plt.rcParams['font.family'] = 'Times New Roman'
       plt.rcParams['font.size'] =25

       data=[value0,value1]

       fig =plt.figure(figsize=(8,8))
       ax1 = fig.subplots()
       ax1.yaxis.set_major_locator(MaxNLocator(5))
       ax1.boxplot(data,whis=(0,100),vert=True,  # 横向きにする
              patch_artist=True,  # 細かい設定をできるようにする
              widths=0.5,  # boxの幅の設定
              boxprops=dict(facecolor='gray',  # boxの塗りつぶし色の設定
                            color='black', linewidth=1),  # boxの枠線の設定
              medianprops=dict(color='black', linewidth=1),  # 中央値の線の設定
              whiskerprops=dict(color='black', linewidth=1),  # ヒゲの線の設定
              capprops=dict(color='black', linewidth=1),  # ヒゲの先端の線の設定
              )
       ax1.set_xticks(ax1.get_xticks())
       ax1.set_xticklabels(['Negative','Positive'])
       ax1.set_ylabel(x_label)

       y_ticks =np.array(ax1.get_yticks())
       ax1.set_yticks(y_ticks)
       ax1.set_yticklabels(np.round(y_ticks))
       # ax1.set_xlabel('Year')
       plt.title(title
       ,fontname='MS Mincho'
       )
       value0=value0*change_num
       value1 =value1*change_num
       
       num_round = 4
       print(year,target[0])
       print('平均値','negative',round(np.mean(value0),num_round),' ','positive',round(np.mean(value1),num_round))
       print('中央値','negative',round(np.median(value0),num_round),' ','positive',round(np.median(value1),num_round))
       print('標準偏差','negative',round(np.std(value0,ddof=1),num_round),' ','positive',round(np.std(value1,ddof=1),num_round))
       print('四分位範囲','negative',round(np.percentile(value0,75)-np.percentile(value0,25),num_round),' ','positive',round(np.percentile(value1,75)-np.percentile(value1,25),num_round))


In [8]:
df2

,DateTime,gap_price[intra-spot](円/kWh),太陽光絶対値誤差合算(実績-想定)(kWh)
66673,2022-01-01 09:00:00,5.91,250631.932962
66674,2022-01-01 09:30:00,3.05,184973.486658
66675,2022-01-01 10:00:00,2.62,284123.186423
66676,2022-01-01 10:30:00,1.51,298702.090738
66677,2022-01-01 11:00:00,7.85,397339.462726
...,...,...,...
79740,2022-09-30 14:30:00,3.08,197858.191143
79741,2022-09-30 15:00:00,4.59,266132.919788
79742,2022-09-30 15:30:00,4.62,227144.453454
79743,2022-09-30 16:00:00,4.29,260739.104841
